In [4]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    ...

In [5]:
# SECTION 1 — Imports + Setup

import json
import os
import datetime
import pandas as pd
from IPython.display import display, Markdown
import ipywidgets as widgets

print("Imports loaded.")

Imports loaded.


In [6]:
# SECTION 2 — Memory System (Auto-create JSON file)

MEMORY_FILE = "memory_bank.json"

# Create file if missing
if not os.path.exists(MEMORY_FILE):
    default_memory = {
        "history": [],
        "last_workout": None,
        "goals": {"mode": "muscle_gain", "target_calories": 2800}
    }
    with open(MEMORY_FILE, "w") as f:
        json.dump(default_memory, f, indent=4)

def read_memory():
    with open(MEMORY_FILE, "r") as f:
        return json.load(f)

def write_memory(data):
    with open(MEMORY_FILE, "w") as f:
        json.dump(data, f, indent=4)

def update_memory(key, value):
    mem = read_memory()
    mem[key] = value
    write_memory(mem)

print("Memory initialized.")

Memory initialized.


In [7]:
# SECTION 3 — Tools

# Daily Logger Tool
def log_day(entry):
    mem = read_memory()
    mem["history"].append(entry)
    write_memory(mem)

# Calculator Tool
def calculate_calories(meal_plan):
    return sum(item["calories"] for item in meal_plan.values())

# Google Search Mock Tool (you can integrate real Google search if needed)
def get_calories_from_google(item):
    fake_database = {
        "eggs": 155, "banana": 105, "chicken breast": 260,
        "rice": 200, "peanut butter": 190, "oats": 300,
        "greek yogurt": 120, "nuts": 180, "paneer": 320
    }
    return fake_database.get(item.lower(), 150)

print("Tools ready.")

Tools ready.


In [8]:
# SECTION 4 — Agents

# Agent 1 — Fitness Coach (Muscle Gain Focus)
def fitness_agent(sleep, energy, mood, last_workout):

    # Decide intensity
    if sleep < 6:
        intensity = "Low"
    elif energy == "High":
        intensity = "High"
    else:
        intensity = "Medium"

    # Muscle rotation (Push → Pull → Legs)
    rotation = ["Push", "Pull", "Legs"]
    if last_workout not in rotation:
        next_w = "Push"
    else:
        idx = rotation.index(last_workout)
        next_w = rotation[(idx + 1) % 3]

    workout = {
        "muscle_group": next_w,
        "intensity": intensity,
        "duration": "50–70 minutes",
        "note": "Focus on progressive overload. Increase weight if possible."
    }

    return workout


# Agent 2 — Meal Planner (2800–3000 kcal for muscle gain)
def meal_planner_agent(intensity):

    base_cals = 2800
    if intensity == "High":
        base_cals += 200

    meals = {
        "breakfast": {"item": "Oats + eggs + banana", "calories": 550},
        "snack1": {"item": "Peanut butter sandwich", "calories": 350},
        "lunch": {"item": "Rice + chicken breast + veggies", "calories": 700},
        "snack2": {"item": "Greek yogurt + nuts", "calories": 300},
        "dinner": {"item": "Roti + paneer/chicken", "calories": 600}
    }

    total = calculate_calories(meals)
    return meals, total


# Agent 3 — Schedule Planner (Default wake-up 7:00 AM)
def scheduler_agent(workout, meals, constraints):

    schedule = [
        ("07:00 AM", "Wake up"),
        ("07:30 AM", "Breakfast"),
        ("09:00 AM", "Deep Work Block"),
        ("11:00 AM", "Short walk / Break"),
        ("12:30 PM", "Lunch"),
        ("02:00 PM", "Work Block"),
        ("04:00 PM", "Snack"),
        ("06:00 PM", f"Workout — {workout['muscle_group']} ({workout['intensity']})"),
        ("08:00 PM", "Dinner"),
        ("10:30 PM", "Reflection / Relax"),
    ]

    if constraints.strip():
        schedule.append(("NOTE", f"User constraints: {constraints}"))

    return schedule


# Agent 4 — Daily Loop (Morning Summary)
def loop_agent_morning(inputs, workout, meals, total_cals, schedule):

    entry = {
        "date": str(datetime.date.today()),
        "morning_inputs": inputs,
        "plan": {
            "workout": workout,
            "meals": meals,
            "total_calories": total_cals,
            "schedule": schedule
        }
    }

    log_day(entry)
    return entry

print("Agents loaded.")

Agents loaded.


In [9]:
# SECTION 5 — UI (ipywidgets)

sleep_input = widgets.IntSlider(value=7, min=4, max=10, description="Sleep Hours")
energy_input = widgets.Dropdown(options=["Low", "Medium", "High"], description="Energy")
mood_input = widgets.Dropdown(options=["Stressed", "Neutral", "Motivated"], description="Mood")
constraints_input = widgets.Textarea(description="Constraints", placeholder="Meetings, travel, etc.")

generate_button = widgets.Button(
    description="Generate Day Plan", 
    button_style="success"
)

display(sleep_input, energy_input, mood_input, constraints_input, generate_button)

print("UI loaded.")

IntSlider(value=7, description='Sleep Hours', max=10, min=4)

Dropdown(description='Energy', options=('Low', 'Medium', 'High'), value='Low')

Dropdown(description='Mood', options=('Stressed', 'Neutral', 'Motivated'), value='Stressed')

Textarea(value='', description='Constraints', placeholder='Meetings, travel, etc.')

Button(button_style='success', description='Generate Day Plan', style=ButtonStyle())

UI loaded.


In [10]:
# SECTION 6 — Bind Generate Button to Agents

output = widgets.Output()
display(output)

def on_generate_clicked(b):
    with output:
        output.clear_output()

        sleep = sleep_input.value
        energy = energy_input.value
        mood = mood_input.value
        constraints = constraints_input.value

        memory = read_memory()
        last_workout = memory.get("last_workout", "Push")

        # Agents
        workout = fitness_agent(sleep, energy, mood, last_workout)
        meals, total_cals = meal_planner_agent(workout["intensity"])
        schedule = scheduler_agent(workout, meals, constraints)

        inputs = {
            "sleep": sleep,
            "energy": energy,
            "mood": mood,
            "constraints": constraints
        }

        entry = loop_agent_morning(inputs, workout, meals, total_cals, schedule)

        # Update memory
        update_memory("last_workout", workout["muscle_group"])

        # Display Output
        display(Markdown("## Your Personalized Daily Plan"))

        display(Markdown("### Workout"))
        display(workout)

        display(Markdown("### Meals"))
        display(meals)
        display(Markdown(f"*Total Calories:* {total_cals} kcal"))

        display(Markdown("### Schedule"))
        for t, task in schedule:
            display(Markdown(f"- *{t}* — {task}"))

generate_button.on_click(on_generate_clicked)

print("Button bound.")

Output()

Button bound.


In [11]:
# SECTION 9 — Observability / Logs

mem = read_memory()
df = pd.DataFrame(mem["history"])
df.tail()

""
